# MON NAIVE BAYES

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('./tweet_emotions.csv')
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [ ]:
df = pd.read_csv('tweet_emotions.csv')
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


Data Preparation

In [4]:
# Hapus data yg kosong
df.dropna()

# Hapus data duplikat
df.drop_duplicates(inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [5]:
for x in df.index:
  if df.loc[x, "sentiment"] == 'empty':
    df.drop(x,inplace=True)

In [ ]:
df.head()

,tweet_id,sentiment,content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...


In [9]:
%pip install BeautifulSoup4 scikit-learn textblob 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/69/8a/cf17d6443f5f537e099be81535a56ab68a473f9393fbffda38cd19899fc8/scikit_learn-1.3.2-cp310-cp310-win_amd64.whl.metadata
  Using cached scikit_learn-1.3.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)
  Obtaining dependency information for soupsieve>1.2 from https://files.pythonhosted.org/packages/4c/f3/038b302fdfbe3be7da016777069f26ceefe11a681055ea1f7817546508e3/soupsieve-2.5-py3-none-any.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/f8/ec/b46756f80e3f4c5f0989f6e4492c2851f156d9c239d554754a3c8cffd4e2/scipy-1.11.4-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 1.6 MB/s eta 0:00:00
  Obtaining 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\CJ\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Multinomial NB

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Load the training data from the CSV file
df_train_gaussian = df

# Preprocess the training data
x = df_train_gaussian['content']
y = df_train_gaussian['sentiment']

# using count vectorizer
vectorizer = CountVectorizer()
x_train_vectorized = vectorizer.fit_transform(x)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(x_train_vectorized, y, test_size=0.2, random_state=42)

# initialize multinomialNB classifier
classifier = MultinomialNB()
classifier.fit(X_train.toarray(), y_train)

# Predict on the test data
y_pred = classifier.predict(X_test.toarray())

# Evaluate the accuracy of the classifier
accuracy = (y_test == y_pred).sum() / len(y_test)
print(f"Accuracy: {accuracy}")


Accuracy: 0.3156349712827058


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Load the training data from the CSV file
df_train_gaussian = df

# Preprocess the training data
x = df_train_gaussian['content']
y = df_train_gaussian['sentiment']

# using count vectorizer
vectorizer = CountVectorizer()
x_train_vectorized = vectorizer.fit_transform(x)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(x_train_vectorized, y, test_size=0.2, random_state=42)

# Initialize the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Train the classifier on the training data
gnb.fit(X_train.toarray(), y_train)

# Predict on the test data
y_pred = gnb.predict(X_test.toarray())

# Evaluate the accuracy of the classifier
accuracy = (y_test == y_pred).sum() / len(y_test)
print(f"Accuracy: {accuracy}")


Accuracy: 0.11742182514358647


In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

titles = ["magics-return-i-can-see-the-spirits", "taking-the-mafia-to-the-magic-world", "ill-surpass-the-mc"]

for title in titles:
    url = f'https://www.lightnovelworld.com/novel/{title}'
    response = requests.get(url, headers=headers)

    data_dict = {'title': '', 'author': '', 'rating': '', 'categories': [], 'comments': [], 'sentiments': []}

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get novel title
        title_element = soup.find('h1', class_='novel-title')
        if title_element:
            data_dict['title'] = title_element.text.strip()

        # Get author information
        author_element = soup.find('span', attrs={'itemprop': 'author'})
        if author_element:
            data_dict['author'] = author_element.text.strip()

        # Get rating information
        meta_rating_element = soup.find('meta', attrs={'itemprop': 'ratingValue'})
        if meta_rating_element:
            data_dict['rating'] = meta_rating_element.get('content', '').strip()

        # Mencari semua elemen <p> di dalam <div> dengan class 'comment-text'
        comment_text_divs = soup.find_all('div', class_='comment-text')

        for comment_text_div in comment_text_divs:
            p_elements = comment_text_div.find_all('p')

            for p_element in p_elements:
                paragraph_text = p_element.text.strip()
                data_dict['comments'].append(paragraph_text)

                # Perform sentiment analysis using TextBlob
                blob = TextBlob(paragraph_text)
                sentiment = 'positive' if blob.sentiment.polarity > 0 else 'negative' if blob.sentiment.polarity < 0 else 'neutral'
                data_dict['sentiments'].append(sentiment)

        # Get Categories
        categories_header = soup.find('h4', string='Categories')
        if categories_header:
            categories_list = []
            ul_element = categories_header.find_next('ul')
            if ul_element:
                categories = ul_element.find_all('li')
                for category in categories:
                    category_text = category.text.strip()
                    categories_list.append(category_text)
                data_dict['categories'] = categories_list

        # Print each comment with its predicted sentiment
        print(data_dict)
        for comment, sentiment in zip(data_dict['comments'], data_dict['sentiments']):
            print(f"Comment: {comment}")
            print(f"Predicted Sentiment: {sentiment}\n")
    else:
        print(f"Gagal mengambil halaman web untuk judul {title}. Kode status:", response.status_code)


{'title': "Magic's Return: I Can See The Spirits", 'author': 'SleepDeprivedSloth', 'rating': '4.0', 'categories': ['Fantasy', 'Action', 'Romance', 'Comedy'], 'comments': ['Any novels with similar romance? Pls reccomend me some...', 'The novel has good romance, pacing, fight scenes, but Vaan feels like a 15 year old.\nWhat 31 year old genocidal mob boss says things like “simp general” and has the patience and ego of a toddler? His *ss would’ve gotten killed years ago.\nIf you can get past the fact the mc is basically an egotistical teenager with no character development yet, the rest of the novel is enjoyable.', 'I can’t get over his personality. He literally attacks people and when they defend themselves he says “my life is so precious how dare he try to take it from me”', 'Well I would argue that isn’t actually as bad as some other actions he takes as most gangsters use self-justification as a way to cope with the crimes they commit. I personally can’t get over how he literally acts l